Improvement of the through time factor


AR change factor card: ARxARxS


S is the variable that controls the standard deviation parameter of the gaussian noise added to the previous day's airway resistance

In [1]:
import src.models.var_builders as var_builders
import src.data.helpers as dh
import src.data.breathe_data as bd
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import src.models.helpers as mh
import src.models.cpts.helpers as cpth
import src.modelling_ar.AR_joint_sampling as model_ar

In [2]:
AR = mh.VariableNode("Airway resistance (%)", 0, 90, 2, prior=None)
AR1 = mh.VariableNode("Airway resistance day 1 (%)", 0, 90, 2, prior=None)
AR2 = mh.VariableNode("Airway resistance day 2 (%)", 0, 90, 2, prior=None)
S = mh.DiscreteVariableNode("AR change factor shape", 2, 10, 2)
S.values

array([ 2,  4,  6,  8, 10])

In [3]:
# Each bin of AR is uniformly distributed
# P(AR2 | AR1 = [0, 2]) is the convolution of a laplace distribution with a uniform distribution

# Let's just take the midbin for now
def laplace_pdf(x, mu, b):
    return 1/(2*b) * np.exp(-np.abs(x - mu)/b)

def gaussian_pdf(x, mu, sigma):
    return 1/(sigma * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((x - mu) / sigma)**2)

y_laplace = laplace_pdf(AR2.midbins, AR1.midbins[10], 5)
y_laplace = y_laplace / np.sum(y_laplace)

y_gaussian = gaussian_pdf(AR2.midbins, AR1.midbins[10], 5)
y_gaussian = y_gaussian / np.sum(y_gaussian)

fig = go.Figure()
fig.add_trace(go.Scatter(x=AR2.midbins, y=y_laplace, mode='markers+lines'))
# fig.add_trace(go.Scatter(x=AR2.midbins, y=y_gaussian, mode='markers+lines'))
fig.show()

In [4]:
# Build CPT

cpt = np.zeros((AR2.card, AR2.card, S.card))

for b_idx, b in enumerate(S.values):
    for i in range(AR1.card):
            p_ar2 = laplace_pdf(AR2.midbins, AR1.midbins[i], b)
            p_ar2 = p_ar2 / np.sum(p_ar2)
            cpt[:, i, b_idx] = p_ar2


In [5]:
cpth.save_cpt([AR, AR, S], cpt, "_shape_factor")